# Introduction
This is a simple collection of classes and functions to price, and find the greeks for, a range of options including European, American and Asian Calls and Puts, and options with more exotic (including path dependent) payoff functions specified by the user. Underlying calculations are performed via implementations of binary trees and MonteCarlo simulations, e.g., see Hull 11th Ed., Ch 21. European and American options are priced on the binomial tree, European and Asian options are priced on the Monte Carlo simulator, the path dependent contracts -priced on the Monte Carlo simulator, do not accomodate early excercise, but are otherwise general.

The next section provides example use cases showing how to construct an option contract's terms, construct the underlying asset, and construct the option from those. Section 1 prices an American Put option on a binomial tree following an example from Hull. Section 2 shows how to obtain the sensitivities (Greeks) of this option, and compares results with RQuantLib(in addition to Hull). Section 3 shows how to specify arbitrary payoffs to price exotic American or European options on the binary tree. Section 4 illustrates the use of the Monte Carlo pricing engine to get the price and greeks of European options, goes on to illustrate its use in valuing path dependent (Asian) options, and then how to specify and value arbitrary path dependent contracts.

# Example Uses 
### Technicalities
If you are running this jupyter notebook with the Xeus-cling kernel installed, (https://xeus-cling.readthedocs.io/en/latest/, https://learnopencv.com/xeus-cling-run-c-code-in-jupyter-notebook/) you will be able to execute each of the following code cells, alter values and re-run etc. Alternatively, you will need the files
- option_pricing.hpp
- option_pricing.cc
- main.cc

in some directory from which you can create the executable `main` with the shell cmd: 
> g++ --std=c++14 option_pricing.cc main.cc -o main
 
[Tech Note: There were technical difficulities at the time of this document with the xeus-cling implementation of the C++17 kernel, reverting to C++14 resulted in fewer issues..]

Running `./main` will then produce all the output of the following Example cells.

### 1) American Put Option on a Binary Tree
We apply the binomial tree to the American Put option, example 21.1 of Hull $11^{\text{th}}$ ed.

`option_pricing.hpp` declares the interface into the functionality, all inside the `namespace OptionPricing`:

In [1]:
// option_pricing.hpp:
#include <functional>
#include <iostream>

namespace OptionPricing {

    class Asset {
    public:
        Asset(double S_o, double q, double sigma);
        Asset(const Asset&) = default;

        double   S_o() const { return _S_o; }
        double     q() const { return _q; }
        double sigma() const { return _sigma; }

    private:
        double _S_o;
        double _q;
        double _sigma;
    };


    class Terms {
    public:
        enum class Style { Euro, Amer, AsianPrc, AsianStrike, AsianExotic };
        enum class Type { Call, Put, Other };
        enum class AvgType { Arith, Geom, Other, None };

        static std::function<void(int, double, double&)> arith_path_accum_fn;
        static std::function<void(int, double, double&)> geom_path_accum_fn;

        Terms(Style, Type, double T, double K, AvgType = AvgType::Arith);
        Terms(Style, Type, double T, AvgType = AvgType::Arith);
        Terms(Style, Type, double T, std::function<double(double)> payoff);
        Terms(Style, Type, double T, std::function<double(double, double)> path_payoff,
            std::function<void(int, double, double&)> path_accum, double init_accum);
        Terms(const Terms&) = default;
        Terms() = delete;

        Style style() const { return _style; }
        Type type() const { return _type; }
        double K() const { return _K; }
        double T() const { return _T; }
        AvgType avgType() const { return _avg_type; }
        std::function<double(double)> payoff_fn() const { return _payoff; }
        std::function<double(double, double)> path_payoff() const { return _path_payoff; }
        std::function<void(int, double, double&)> path_accum() const { return _path_accum_fn; }
        double init_accum() const { return _init_accum; }

        double payoff(double S) const { return _payoff(S); }

    private:
        const Style _style;
        const Type _type;
        const double _T;
        const double _K;
        AvgType _avg_type;
        std::function<double(double)> _payoff{};
        std::function<double(double, double)> _path_payoff{};
        std::function<void(int, double, double&)> _path_accum_fn{};
        double _init_accum{ 0.0 };
    };


    class Option {
    public:
        Option(const Terms&, const Asset&);

        double btree_prc(int N_steps, double r, bool get_greeks);
        double MC_prc(int N_sims, int N_steps, double r, bool get_greeks,
            bool antithetic, double& sd_prc);


        struct Greeks {
            double delta{ 0 };
            double gamma{ 0 };
            double theta{ 0 };
            double   rho{ 0 };
            double  vega{ 0 };
            double delta2{ 0 };
            double theta2{ 0 };
        };

        Terms  terms() const { return _terms; }
        Asset  asset() const { return _asset; }
        double price() const { return _prc; }
        Greeks all_greeks() const { return _greeks; }
        double delta() const { return _greeks.delta; }
        double gamma() const { return _greeks.gamma; }
        double theta() const { return _greeks.theta; }
        double   rho() const { return _greeks.rho; }
        double  vega() const { return _greeks.vega; }
        double delta2() { return _greeks.delta2; }
        double theta2() { return _greeks.theta2; }

    private:
        const Terms _terms;
        const Asset _asset;
        double _prc;
        Greeks _greeks;
    };

    double btree_prc_opt(Option&, int N_steps, double r, bool greeks);

    double btree_prc_terms_assets(const Terms&, const Asset&, int N_steps, double r,
        Option::Greeks*); // Note user danger here.

    double btree_prc_raw(int N_steps, double r,
        Terms::Style style, double T, std::function<double(double)>,
        double S_o, double q, double sigma, Option::Greeks*);


    double MC_prc_opt(Option&, int N_sims, int N_steps, double r, bool greeks,
        bool antithetic, double& sd_prc);

    double MC_prc_terms_assets(const Terms&, const Asset&, int N_sims, int N_steps, double r,
        Option::Greeks*, bool antithetic, double& sd_prc);

    double MC_prc_raw(int N_sims, int N_steps, double r,
        Terms::Style style, double T, std::function<double(double)> payoff,
        std::function<double(double, double)> path_payoff,
        std::function<void(int, double, double&)> path_accum, double init_accum,
        double S_o, double q, double sigma, Option::Greeks*,
        bool antithetic, double& sd_prc);


    std::ostream& operator<< (std::ostream&, const Asset&);
    std::ostream& operator<< (std::ostream&, const Terms&);
    std::ostream& operator<< (std::ostream&, const Option&);

}


`option_pricing.cc` defines the implementation of anything not already implemented(those were inline functions) in `option_pricing.hpp`:

In [2]:
// option_pricing.cc:
// #include "option_pricing.hpp"
// #include "fexcpt.hpp"
#include <algorithm>
#include <cmath>
#include <limits>
#include <iostream>
#include <random>


namespace OptionPricing {
    using std::exp;
    using std::sqrt;
    
    Asset::Asset(double S_o, double q, double sigma)
        : _S_o{ S_o }, _q{ q }, _sigma{ sigma } { }


    Terms::Terms(Style s, Type t, double T, double K, AvgType avg)
        // Set terms of Euro, Amer, or AsianPrc Call or Put, each of which define their 
        // _payoff(S) in terms of the strike K in the usual vanilla way.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ K }, _avg_type{ avg } {
        if (t == Type::Other || s == Style::AsianStrike || s == Style::AsianExotic
            || avg == AvgType::Other || _K < 0 || _T <= 0) {
            throw std::invalid_argument(
                "Must provide Euro, Amer or AsianPrc Call or Put with valid K and T.");
        }
        if (s == Style::AsianPrc && avg != AvgType::Arith && avg != AvgType::Geom)
            throw std::invalid_argument("AsianPrc must do an Arith or Geom path price average.\n \
                                         -Use AsianExotic to specify Other path price avgeraging.");
        if (_type == Type::Call)
            _payoff = [K](double S) { return std::max(S - K, 0.0); };
        else
            _payoff = [K](double S) { return std::max(K - S, 0.0); };
        if (_style == Style::AsianPrc) {
            if (_avg_type == AvgType::Arith) {
                _path_accum_fn = arith_path_accum_fn;
                _init_accum = 0.0;
            } else {
                _path_accum_fn = geom_path_accum_fn;
                _init_accum = 1.0;
            }
        } else  // Euro and Amer have no path averaging:
            _avg_type = AvgType::None;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor1: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, AvgType avg)
        // Set terms of AsianStrike. Like AsianPrc, must be a Call or Put and uses only Arith or 
        // Geom path price averaging, -Use AsianExotic to specify other path averaging.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ avg } {
        if (s != Style::AsianStrike || t == Type::Other ||
            (avg != AvgType::Arith && avg != AvgType::Geom) || _T <= 0) {
            throw std::invalid_argument("Must provide AsianStrike Call or Put with valid T\n \
             and can only do Arith or Geom path price average:\n \
              -Use AsianExotic to specify Other path price averaging.");
        }
        if (_type == Type::Call)
            _path_payoff = [](double K_avg, double S) { return std::max(S - K_avg, 0.0); };
        else
            _path_payoff = [](double K_avg, double S) { return std::max(K_avg - S, 0.0); };

        if (_avg_type == AvgType::Arith) {
            _path_accum_fn = arith_path_accum_fn;
            _init_accum = 0.0;
        } else {
            _path_accum_fn = geom_path_accum_fn;
            _init_accum = 1.0;
        }
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor2: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, std::function<double(double)> payoff)
        // Set terms of Euro or Amer option that is not Call or Put (i.e. is Other)
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ AvgType::None } {
        if ((s != Style::Amer && s != Style::Euro) || t != Type::Other || _T <= 0) {
            throw std::invalid_argument("Must provide Euro or Amer with valid T\n \
             and provide payoff function of this exotic(Type::Other) Terms object.");
        }
        _payoff = payoff;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor3: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, std::function<double(double, double)> path_payoff,
        std::function<void(int, double, double&)> path_accum_fn, double init_accum)
        // Set terms of AsianExotic option. 
        // Both path_payoff and path_accum_fn (along with init_accum) need to be specified.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ AvgType::Other } {
        if (s != Style::AsianExotic || t != Type::Other || _T <= 0) {
            throw std::invalid_argument("Must provide AsianExotic with valid T\n \
             path_payoff function of this exotic(Type::Other) Terms object.");
        }
        _path_payoff = path_payoff;
        _path_accum_fn = path_accum_fn;
        _init_accum = init_accum;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor4: " << e.what() << std::endl;
        throw;
    }


    std::function<void(int, double, double&)>
        Terms::arith_path_accum_fn { [](int j, double s, double& cum) {
        cum += (s - cum) / j;
    } };
    std::function<void(int, double, double&)>
        Terms::geom_path_accum_fn = [](int j, double s, double& cum) {
        cum *= pow(s / cum, 1.0 / j);
    };

    Option::Option(const Terms& t, const Asset& a)
        : _terms{ t }, _asset{ a }, _prc{ -1 }, _greeks{} {  }


    double Option::btree_prc(int N_steps, double r, bool get_greeks) {
        Greeks* g_ptr{ get_greeks ? &_greeks : nullptr };
        _prc = btree_prc_terms_assets(_terms, _asset, N_steps, r, g_ptr);
        return _prc;
    }

    double btree_prc_opt(Option& opt, int N_steps, double r, bool greeks) {
        return opt.btree_prc(N_steps, r, greeks);
    }

    double btree_prc_terms_assets(const Terms& t, const Asset& a, int N_steps, double r,
        Option::Greeks* g_ptr) {
        if (t.style() != Terms::Style::Amer && t.style() != Terms::Style::Euro) {
            throw std::invalid_argument("In btree_prc_raw, Bad Terms::Style.");
        }
        return btree_prc_raw(N_steps, r, t.style(), t.T(), t.payoff_fn(),
            a.S_o(), a.q(), a.sigma(), g_ptr);
    }

    double btree_prc_raw(int N, double r,
        Terms::Style style, double T, std::function<double(double)> payoff,
        double S_o, double q, double sigma, Option::Greeks* g_ptr) {

        if (style != Terms::Style::Amer && style != Terms::Style::Euro) {
            throw std::invalid_argument("In btree_prc_raw, Bad Terms::Style.");
        }

        double dt = T / N;
        double u = exp(sigma * sqrt(dt));
        double u2 = u * u;
        double d = 1. / u;
        double a = exp((r - q) * dt);
        double p = (a - d) / (u - d);
        double df = exp(-r * dt);

        struct Node {
            double s{ 0 };
            double f{ 0 };
        };
        int len_nodes = N + 1;
        Node* nodes = new Node[len_nodes]{};

        // Set s and f for final time step:
        nodes[0].s = S_o * std::pow(u, -N);
        nodes[0].f = payoff(nodes[0].s);
        for (int j = 1; j < N + 1; ++j) {
            if (j == N / 2) {  // Take advantage to tie node's s value to known value at middles:
                if (N % 2)     // Odd N, we're one step below middle:
                    nodes[j].s = S_o / u;
                else           // Even N, we're at the middle, so s == S_o:
                    nodes[j].s = S_o;
            } else {
                nodes[j].s = nodes[j - 1].s * u2;
            }
            nodes[j].f = payoff(nodes[j].s);
        }

        std::function<double(int)> step_back_Payoff;
        if (style == Terms::Style::Euro)
            step_back_Payoff = [p, df, nodes](int idx) {
            return df * (p * nodes[idx + 1].f + (1 - p) * nodes[idx].f); };
        else
            step_back_Payoff = [p, df, payoff, nodes](int idx) {
            return std::max(payoff(nodes[idx].s), df * (p * nodes[idx + 1].f + (1 - p) * nodes[idx].f)); };

        // Backstep from time i=N-1 to i=0, filling in nodes j=0 to j=i for each:
        double s1, s2, s3, s4, s5;  // Needed to save some values at i=1, 2 for greeks calculations.
        double f1, f2, f3, f4, f5;
        for (int i = N - 1; i >= 0; --i) {
            for (int j = 0; j <= i; ++j) {
                if (j == i / 2) {  // As above, tie to known values.
                    if (i % 2)     // Potential speed up: break j-loop into before and after middle..
                        nodes[j].s = S_o / u;
                    else
                        nodes[j].s = S_o;
                } else {
                    nodes[j].s = nodes[j].s * u;
                }
                nodes[j].f = step_back_Payoff(j);  // flag_Payoffe("Underflow for fine grid: Ok", false);
// see https://stackoverflow.com/questions/46611633/can-the-floating-point-status-flag-fe-underflow-set-when-the-result-is-not-sub-n
            }
            if (i <= 2) {  // Save some values to calculate delta, gamma, theta:
                if (i == 1) {
                    s1 = nodes[0].s;
                    s2 = nodes[1].s;
                    f1 = nodes[0].f;
                    f2 = nodes[1].f;
                }
                if (i == 2) {
                    s3 = nodes[0].s;
                    s4 = nodes[1].s;
                    s5 = nodes[2].s;
                    f3 = nodes[0].f;
                    f4 = nodes[1].f;
                    f5 = nodes[2].f;
                }
            }
        }

        double f0 = nodes[0].f;

        if (g_ptr) {
            g_ptr->delta = (f2 - f1) / (s2 - s1);
            g_ptr->gamma = 2 * ((f5 - f4) / (s5 - s4) - (f4 - f3) / (s4 - s3)) / (s5 - s3);
            g_ptr->theta = (f4 - f0) / (2 * dt);

            double incr = .0001;
            g_ptr->rho = (btree_prc_raw(N, r + incr, style, T, payoff, S_o, q, sigma, nullptr) - f0) / incr;
            g_ptr->vega = (btree_prc_raw(N, r, style, T, payoff, S_o, q, sigma + incr, nullptr) - f0) / incr;
            //g_ptr->delta2 = (btree_prc_raw(N, r, style, T, F, S_o + incr, q, sigma, nullptr) - f0) / incr;
            //g_ptr->theta2 = (btree_prc_raw(N, r, style, T - incr, F, S_o, q, sigma, nullptr) - f0) / incr;
        }

        delete[] nodes;
        return f0;
    }


    double Option::MC_prc(int N_sims, int N_steps, double r, bool get_greeks, bool antithetic, double& sd_prc) {
        Greeks* g_ptr{ get_greeks ? &_greeks : nullptr };
        _prc = MC_prc_terms_assets(_terms, _asset, N_sims, N_steps, r, g_ptr, antithetic, sd_prc);
        return _prc;
    }

    double MC_prc_opt(Option& opt, int N_sims, int N_steps, double r, bool greeks, bool antithetic, double& sd_prc) {
        return opt.MC_prc(N_sims, N_steps, r, greeks, antithetic, sd_prc);
    }

    double MC_prc_terms_assets(const Terms& t, const Asset& a, int N_sims, int N_steps, double r,
        Option::Greeks* g_ptr, bool antithetic, double& sd_prc) {
        if (t.style() == Terms::Style::Amer) {
            throw std::invalid_argument("In MC_prc_terms_assets, Can't price Amer option via MC.");
        }
        return MC_prc_raw(N_sims, N_steps, r, t.style(), t.T(), t.payoff_fn(), t.path_payoff(),
            t.path_accum(), t.init_accum(), a.S_o(), a.q(), a.sigma(), g_ptr, antithetic, sd_prc);
    }

    double MC_prc_raw(int N_sims, int N_steps, double r, Terms::Style style, double T, std::function<double(double)> payoff,
        std::function<double(double, double)> path_payoff, std::function<void(int, double, double&)> path_accum_fn, double init_accum,
        double S_o, double q, double sigma, Option::Greeks* g_ptr, bool antithetic, double& sd_mean_PV) {

        if (style == Terms::Style::Amer) {
            throw std::invalid_argument("In MC_prc_raw, no Amer terms allowed. \n");
        }

        double g = r - q - sigma * sigma / 2;  // risk neutral growth rate of lnS.
        double df = exp(-r * T);

        // long unsigned int seed = 1;    // Replace next 2 lines with these 2 for reproducing random 
        // static std::mt19937 gen{seed}; // bit streams for testing.
        std::random_device rd;
        static std::mt19937 gen{ rd() };
        std::normal_distribution<> norm{ };

        double cumm_P{ 0.0 };
        double ssq{ 0.0 };
        if (style == Terms::Style::Euro) {
            // Can by-pass path stepping and just simulate final risk-neutral asset price at time T:
            double sd = sigma * sqrt(T);

            if (!g_ptr) {  // nullptr, skip greeks:
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = payoff(S_o * exp(g * T + eps * sd));
                        cumm_P += P;
                        ssq += P * P;
                    }
                } else {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = (payoff(S_o * exp(g * T + eps * sd)) +
                            payoff(S_o * exp(g * T - eps * sd))) / 2;
                        cumm_P += P;
                        ssq += P * P;
                    }
                }
            } else {      // valid g_ptr, get greeks:
                double incr = 0.01;
                double S_o_up{ (1. + incr) * S_o }, S_o_dn{ (1. - incr) * S_o }, g_rho_up{ g + incr * r };
                double cumm_delta_up{ 0 }, cumm_delta_dn{ 0 }, cumm_rho_up{ 0 }, cumm_theta_up{ 0 }, cumm_vega_up{ 0 };
                double sigma_up{ (1 + incr) * sigma };
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = payoff(S_o * exp(g * T + eps * sd));
                        cumm_P += P;
                        ssq += P * P;
                        cumm_delta_up += payoff(S_o_up * exp(g * T + eps * sd));
                        cumm_delta_dn += payoff(S_o_dn * exp(g * T + eps * sd));
                        cumm_rho_up += payoff(S_o * exp(g_rho_up * T + eps * sd));
                        cumm_theta_up += payoff(S_o * exp(g * (1 - incr) * T + eps * sigma * sqrt((1 - incr) * T)));
                        cumm_vega_up += payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T + eps * sigma_up * sqrt(T)));
                    }
                } else {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = (payoff(S_o * exp(g * T + eps * sd)) +
                            payoff(S_o * exp(g * T - eps * sd))) / 2;
                        cumm_P += P;
                        ssq += P * P;
                        cumm_delta_up += (payoff(S_o_up * exp(g * T + eps * sd)) +
                            payoff(S_o_up * exp(g * T - eps * sd))) / 2;
                        cumm_delta_dn += (payoff(S_o_dn * exp(g * T + eps * sd)) +
                            payoff(S_o_dn * exp(g * T - eps * sd))) / 2;
                        cumm_rho_up += (payoff(S_o * exp(g_rho_up * 
                        T + eps * sd)) +
                            payoff(S_o * exp(g_rho_up * T - eps * sd))) / 2;
                        cumm_theta_up += (payoff(S_o * exp(g * (1 - incr) * T + eps * sigma * sqrt((1 - incr) * T))) +
                            payoff(S_o * exp(g * (1 - incr) * T - eps * sigma * sqrt((1 - incr) * T)))) / 2;
                        cumm_vega_up += (payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T + eps * sigma_up * sqrt(T))) +
                            payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T - eps * sigma_up * sqrt(T)))) / 2;
                    }
                }
                /* By linearity these greeks have been calculated once in terms of the accumulated
                   values. To estimate their stddevs, as we do for price, they would need to be moved inside
                   the loop over N_sims to and the squares of their values accumulated. The code as 
                   written is much more compact/readable...
                   */
                double delta_up = df * (cumm_delta_up - cumm_P) / (N_sims * incr * S_o);
                double delta_dn = df * (cumm_P - cumm_delta_dn) / (N_sims * incr * S_o);
                g_ptr->delta = (delta_dn + delta_up) / 2.0;
                g_ptr->gamma = (delta_up - delta_dn) / (incr * S_o);
                g_ptr->rho = df * (exp(-incr * r * T) * cumm_rho_up - cumm_P) / (N_sims * incr * r);
                g_ptr->theta = df * (exp(r * incr * T) * cumm_theta_up - cumm_P) / (N_sims * incr * T);
                g_ptr->vega = df * (cumm_vega_up - cumm_P) / (N_sims * incr * sigma);
            }
        } else { // Asian must simulate each time step:
            double dt = T / N_steps;
            double sd = sigma * sqrt(dt);
            double path_mean;
            double S;
            if (!g_ptr) { // nullptr -skip greeks
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        S = S_o;
                        path_mean = init_accum;
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);  // Could use (S_prev + S) / 2.
                        }
                        double P = (style == Terms::Style::AsianPrc) ? payoff(path_mean) : path_payoff(path_mean, S);
                        cumm_P += P;
                        ssq += P * P;
                    }
                } else { // Perform antithetic sampling:
                    // Introduce the antithetic path 'accumulators':
                    double path_mean_anti;
                    double S_anti;
                    for (int i = 1; i <= N_sims; ++i) {
                        // Initialize path accumulators prior to this(i) sim/run:
                        path_mean = init_accum;
                        path_mean_anti = init_accum;
                        S = S_o;
                        S_anti = S_o;
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            S_anti *= exp(g * dt - eps * sd);
                            path_accum_fn(j + 1, S_anti, path_mean_anti);
                        }
                        double P = (style == Terms::Style::AsianPrc) ? (payoff(path_mean) + payoff(path_mean_anti)) / 2
                            : (path_payoff(path_mean, S) + path_payoff(path_mean_anti, S_anti)) / 2;
                        cumm_P += P;
                        ssq += P * P;
                    }
                }
            } else { // valid g_ptr, get greeks:
                double incr = 0.01;
                double sigma_up{ (1 + incr) * sigma };
                // Initialize accumulators before the N_sims:
                double cumm_delta_up{ 0 }, cumm_delta_dn{ 0 }, cumm_rho_up{ 0 },
                    cumm_theta_up{ 0 }, cumm_vega_up{ 0 };
                // S, path_mean and the following 7 path accumulators get reset for each of the N_sims by init_sim[&]():
                double S_rho_up, S_theta_up, S_vega_up;
                double path_mean_up, path_mean_dn;
                double pm_rho_up, pm_vega_up, pm_theta_up;
                if (!antithetic) {
                    std::function<void()> init_sim{ [&] () {
                        S = S_o; S_rho_up = S_o; S_theta_up = S_o; S_vega_up = S_o;
                        path_mean = init_accum; path_mean_up = init_accum; path_mean_dn = init_accum;
                        pm_rho_up = init_accum; pm_vega_up = init_accum;
                    } };
                    for (int i = 0; i < N_sims; ++i) {
                        init_sim();  // reset this sim run accumulators.
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            path_accum_fn(j + 1, S * (1 + incr), path_mean_up);
                            path_accum_fn(j + 1, S * (1 - incr), path_mean_dn);
                            S_rho_up *= exp((g + incr * r) * dt + eps * sd);
                            path_accum_fn(j + 1, S_rho_up, pm_rho_up);
                            S_vega_up *= exp((r - q - sigma_up * sigma_up / 2) * dt + eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up, pm_vega_up);
                            if (j == N_steps - 2) { // special treatment -conditional has execution cost time(could put outside loop).
                                pm_theta_up = path_mean;
                                S_theta_up = S;
                            }
                        }
                        double P{};
                        if (style == Terms::Style::AsianPrc) {
                            P = payoff(path_mean);
                            cumm_delta_up += payoff(path_mean_up);
                            cumm_delta_dn += payoff(path_mean_dn);
                            cumm_rho_up += payoff(pm_rho_up);
                            cumm_vega_up += payoff(pm_vega_up);
                            cumm_theta_up += payoff(pm_theta_up);
                        } else {
                            P = path_payoff(path_mean, S);
                            cumm_delta_up += path_payoff(path_mean_up, S * (1 + incr));
                            cumm_delta_dn += path_payoff(path_mean_dn, S * (1 - incr));
                            cumm_rho_up += path_payoff(pm_rho_up, S);
                            cumm_vega_up += path_payoff(pm_vega_up, S);
                            cumm_theta_up += path_payoff(pm_theta_up, S_theta_up);
                        }
                        cumm_P += P;
                        ssq += P * P;
                    }
                } else {  // antithetic:
                    // Introduce the antithetic path 'accumulators':
                    double path_mean_anti;
                    double S_anti;
                    double S_rho_up_anti, S_theta_up_anti, S_vega_up_anti;
                    double path_mean_up_anti, path_mean_dn_anti;
                    double pm_rho_up_anti, pm_vega_up_anti, pm_theta_up_anti;

                    // init_sim_antith() called to initialize path accumulators prior to each of the N_sims:
                    std::function<void()> init_sim_antith{ [&] () {
                        S = S_o; S_rho_up = S_o; S_theta_up = S_o; S_vega_up = S_o;
                        path_mean = init_accum; path_mean_up = init_accum; path_mean_dn = init_accum;
                        pm_rho_up = init_accum; pm_vega_up = init_accum;
                        S_anti = S_o; S_rho_up_anti = S_o; S_theta_up_anti = S_o; S_vega_up_anti = S_o;
                        path_mean_anti = init_accum; path_mean_up_anti = init_accum; path_mean_dn_anti = init_accum;
                        pm_rho_up_anti = init_accum; pm_vega_up_anti = init_accum;
                    } };

                    for (int i = 0; i < N_sims; ++i) {
                        init_sim_antith();
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            path_accum_fn(j + 1, S * (1 + incr), path_mean_up);
                            path_accum_fn(j + 1, S * (1 - incr), path_mean_dn);
                            S_rho_up *= exp((g + incr * r) * dt + eps * sd);
                            path_accum_fn(j + 1, S_rho_up, pm_rho_up);
                            S_vega_up *= exp((r - q - sigma_up * sigma_up / 2) * dt + eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up, pm_vega_up);
                            if (j == N_steps - 2) { // special treatment -conditional has execution cost time(could put outside loop).
                                pm_theta_up = path_mean;
                                S_theta_up = S;
                            }
                            // Same for antithetic path:
                            S_anti *= exp(g * dt - eps * sd);
                            path_accum_fn(j + 1, S_anti, path_mean_anti);
                            path_accum_fn(j + 1, S_anti * (1 + incr), path_mean_up_anti);
                            path_accum_fn(j + 1, S_anti * (1 - incr), path_mean_dn_anti);
                            S_rho_up_anti *= exp((g + incr * r) * dt - eps * sd);
                            path_accum_fn(j + 1, S_rho_up_anti, pm_rho_up_anti);
                            S_vega_up_anti *= exp((r - q - sigma_up * sigma_up / 2) * dt - eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up_anti, pm_vega_up_anti);
                            if (j == N_steps - 2) {
                                pm_theta_up_anti = path_mean_anti;
                                S_theta_up_anti = S_anti;
                            }
                        }
                        // Accumulate for this(i) sim/run:
                        double P{};
                        if (style == Terms::Style::AsianPrc) {
                            P = (payoff(path_mean) + payoff(path_mean_anti)) / 2;  // averaging antithetic path results.
                            cumm_delta_up += (payoff(path_mean_up) + payoff(path_mean_up_anti)) / 2;
                            cumm_delta_dn += (payoff(path_mean_dn) + payoff(path_mean_dn_anti)) / 2;
                            cumm_rho_up += (payoff(pm_rho_up) + payoff(pm_rho_up_anti)) / 2;
                            cumm_vega_up += (payoff(pm_vega_up) + payoff(pm_vega_up_anti)) / 2;
                            cumm_theta_up += (payoff(pm_theta_up) + payoff(pm_theta_up_anti)) / 2;
                        } else {
                            P = (path_payoff(path_mean, S) + path_payoff(path_mean_anti, S_anti)) / 2;
                            cumm_delta_up += (path_payoff(path_mean_up, S * (1 + incr)) + path_payoff(path_mean_up_anti, S_anti * (1 + incr))) / 2;
                            cumm_delta_dn += (path_payoff(path_mean_dn, S * (1 - incr)) + path_payoff(path_mean_dn_anti, S_anti * (1 - incr))) / 2;
                            cumm_rho_up += (path_payoff(pm_rho_up, S_rho_up) + path_payoff(pm_rho_up_anti, S_rho_up_anti)) / 2;
                            cumm_vega_up += (path_payoff(pm_vega_up, S_vega_up) + path_payoff(pm_vega_up_anti, S_vega_up_anti)) / 2;
                            cumm_theta_up += (path_payoff(pm_theta_up, S_theta_up) + path_payoff(pm_theta_up_anti, S_theta_up_anti)) / 2;
                        }
                        cumm_P += P;
                        ssq += P * P;
                    }

                }
                double delta_up = df * (cumm_delta_up - cumm_P) / (N_sims * incr * S_o);
                double delta_dn = df * (cumm_P - cumm_delta_dn) / (N_sims * incr * S_o);
                g_ptr->delta = (delta_dn + delta_up) / 2.0;
                g_ptr->gamma = (delta_up - delta_dn) / (incr * S_o);
                g_ptr->rho = df * (exp(-incr * r * T) * cumm_rho_up - cumm_P) / (N_sims * incr * r);
                g_ptr->theta = df * (exp(r * dt) * cumm_theta_up - cumm_P) / (N_sims * dt); // uses dt, not incr*T.
                g_ptr->vega = df * (cumm_vega_up - cumm_P) / (N_sims * incr * sigma);
            }
        }

        double mean_P = cumm_P / N_sims;
        sd_mean_PV = df * sqrt((ssq - N_sims * mean_P * mean_P)) / (N_sims - 1);
        double mean_PV = df * mean_P;

        /* // These had much higher variance than by using same sample paths for up/down estimates..
                if (g_ptr) {
                    double incr = 0.01;
                    double delta_up = (MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        (1 + incr) * S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * S_o);
                    double delta_down = (mean_P - MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        (1 - incr) * S_o, q, sigma, nullptr, antithetic, sd_mean_P)) / (incr * S_o);
                    g_ptr->delta = (delta_down + delta_up) / 2.0;
                    g_ptr->gamma = (delta_up - delta_down) / (incr * S_o);
                    g_ptr->rho = (MC_prc_raw(N_sims, N_steps, (1 + incr) * r, style, T, payoff, path_payoff, path_accum_fn,
                        S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * r);
                    g_ptr->vega = (MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        S_o, q, (1 + incr) * sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * sigma);
                    g_ptr->theta = (MC_prc_raw(N_sims, N_steps, r, style, (1 - incr) * T, payoff, path_payoff, path_accum_fn,
                        S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * T);
                }
        */
        return mean_PV;
    }

    std::ostream& operator<< (std::ostream& os, const Terms& t) {
        switch (t.style()) {
        case Terms::Style::Euro: os << "European ";  break;
        case Terms::Style::Amer: os << "American ";  break;
        case Terms::Style::AsianPrc: os << "AsianPrc ";  break;
        case Terms::Style::AsianStrike: os << "AsianStrike ";  break;
        case Terms::Style::AsianExotic: os << "AsianExotic ";  break;
            // case Terms::Style::Other: os << "Other "; break;
        default: os.setstate(std::ios_base::failbit);
        }
        if (t.style() == Terms::Style::AsianPrc || t.style() == Terms::Style::AsianStrike
            || t.style() == Terms::Style::AsianExotic) {
            switch (t.avgType()) {
            case Terms::AvgType::Arith: os << "Arithmetic ";  break;
            case Terms::AvgType::Geom: os << "Geometric ";  break;
            case Terms::AvgType::Other: os << "Other ";  break;
            case Terms::AvgType::None: os << "None ";  break;
            default: os.setstate(std::ios_base::failbit);
            }
        }


        switch (t.type()) {
        case Terms::Type::Call: os << "Call  ";  break;
        case Terms::Type::Put: os << "Put   ";  break;
        case Terms::Type::Other: os << "Other   "; break;
        default: os.setstate(std::ios_base::failbit);
        }
        switch (t.style()) {
        case Terms::Style::Euro:
        case Terms::Style::Amer:
        case Terms::Style::AsianPrc: os << t.K();  break;
        default: os << '-';
        }
        os << ' ' << t.T();
        return os;
    }

    std::ostream& operator<< (std::ostream& os, const Asset& a) {
        os << a.S_o() << "  " << a.q() << "  " << a.sigma();
        return os;
    }

    std::ostream& operator<< (std::ostream& os, const Option& opt) {
        os << opt.terms() << ' ' << opt.asset();
        return os;
    }
}

Now that the OptionPricing namespace is in the Xeus-cling kernel(upon executing cells 1 and 2), we begin to execute some code snippets extracted from `main.cc`.
The option contract's terms are kept separate from the underlying asset it depends on. We first construct `terms` to represent an American Put with strike price `K` of \$50.00, and time to expiration `T` of 5 months. The `underlying` stock is constructed with current price `S_o` of $50.00, dividend yield `q` of 0, and volatility `sigma` of 0.40. The `amer_put` `Option` is constructed from these:


In [3]:
// Create the American Put option from its terms and underlying Asset:
using namespace OptionPricing;
double K = 50.0, T = 5/12.;
Terms terms{ Terms::Style::Amer, Terms::Type::Put, T, K };
double S_o = 50.0, q = 0.0, sigma = 0.40;
Asset underlying{ S_o, q, sigma };
Option amer_put(terms, underlying);

Once `terms` and `underlying` have been constructed, their parameters remain constant and can be obtained through 'getter' functions. For example we can see `terms` strike price by calling its public member function `K()`:

In [4]:
terms.K()

50.000000

[Tech Note: Leaving the ';' off the end of a statement prints the value into the Jupyter notebook session.]

Other public member 'getter' functions are declared within `class Terms` in `namespace OptionPricing` in `option_pricing.hpp`. Likewise `OptionPricing::Asset` and `OptionPricing::Option` objects have some 'getters' for public use without breaking them.



In [5]:
// Now price the option on a binary tree:
using namespace std;
int N_steps = 5;
double r = 0.10;
bool get_greeks = false;
amer_put.btree_prc(N_steps, r, get_greeks);
cout << "Hull, Example 21.1:\n"
     << "Binary tree pricing of option:\n"
     << "r   Style    Type  K   T       S_o q sigma\n"
     << r << ' ' << amer_put << '\n'
     << "N  price:\n"
     << N_steps << ' ' << amer_put.price() << endl;
for (int N_steps : {30, 50, 100, 500}) {
    amer_put.btree_prc(N_steps, r, get_greeks);
    cout << N_steps << ' ' << amer_put.price() << endl;
}


Hull, Example 21.1:
Binary tree pricing of option:
r   Style    Type  K   T       S_o q sigma
0.1 American Put   50 0.416667 50  0  0.4
N  price:
5 4.48846
30 4.26343
50 4.27202
100 4.27806
500 4.28302


#### Dependence of price on number of time steps, N_steps:
As shown in Fig. 21.4 of Hull, the prices obtained from the binomial trees formed using odd/even numbers of time steps, `N_steps`, form decreasing/increasing sequences that bound and converge to a limit as `N_steps` increases:

In [6]:
// Replicate Fig 21.4, price fluctuations and convergence as N_steps increases:
constexpr int NN{51};
double prcs[NN]{ 0 };
cout << "\n Fig. 21.4: N vs Price:\nN  Price:\n";
for (int N_steps = 2; N_steps < NN; ++N_steps) {
     prcs[N_steps] = amer_put.btree_prc(N_steps, r, get_greeks);
     cout << N_steps << ' ' << prcs[N_steps] << endl;
}


 Fig. 21.4: N vs Price:
N  Price:
2 3.98935
3 4.64408
4 4.13793
5 4.48846
6 4.17447
7 4.44045
8 4.20815
9 4.40411
10 4.22008
11 4.38319
12 4.23216
13 4.36997
14 4.24069
15 4.35752
16 4.24495
17 4.34868
18 4.24894
19 4.34199
20 4.25237
21 4.33634
22 4.25532
23 4.33162
24 4.25771
25 4.3282
26 4.26
27 4.32525
28 4.2619
29 4.32243
30 4.26343
31 4.31987
32 4.26459
33 4.31768
34 4.2657
35 4.31585
36 4.26681
37 4.31427
38 4.26788
39 4.31279
40 4.26876
41 4.31142
42 4.26948
43 4.31013
44 4.27014
45 4.30908
46 4.27087
47 4.30813
48 4.27149
49 4.30717
50 4.27202


### 2) Option Sensitivities, the Greeks:
The sensitivity of the option price to its underlying parameters can be estimated by setting the `get_greeks` argument of the `btree_prc` member function to `true`:
(the details of *how* are among those discussed below.)

In [7]:
// Replicate Example 21.2, Estimates of delta, gamma, theta, rho and vega on amer_put:
N_steps = 50;
get_greeks = true;  // calculate the greeks sensitivities.
amer_put.btree_prc(N_steps, r, get_greeks);
cout << "\n Hull, Example 21.2, Greeks found with binary pricing tree with N_steps: " << N_steps << '\n'
          << "r   Style    Class  K   T     S_o q sigma\n"
          << r << ' ' << amer_put << ":\n"
          << "price: " << amer_put.price() << '\n'
     << "delta: " << amer_put.delta() << '\n' << "gamma: " << amer_put.gamma() << '\n'
     << "vega: " << amer_put.vega() << '\n' << "theta: " << amer_put.theta() << '\n'
     << "rho: "  << amer_put.rho() << endl;


 Hull, Example 21.2, Greeks found with binary pricing tree with N_steps: 50
r   Style    Class  K   T     S_o q sigma
0.1 American Put   50 0.416667 50  0  0.4:
price: 4.27202
delta: -0.414933
gamma: 0.0337955
vega: 12.2933
theta: -4.25689
rho: -7.23241


(Note that Hull uses different units for theta($\$ / day$ vs. $\$ / year$), rho and vega($\$ / \%$ vs $\$ / unit$), with conversion factors of 365 and 100.)

As another point of contact, RQuantLib gives:
```
 $ R
> library(RQuantLib)
RQuantLib 0.4.15 built with QuantLib version 1.25. See https://www.quantlib.org for more on QuantLib.
> AmericanOption(type="put", underlying=50, strike=50, dividendYield=0.0, 
riskFreeRate=0.1, maturity=5/12, volatility=0.4, engine="CrankNicolson")
Concise summary of valuation for AmericanOption 
  value   delta   gamma    vega   theta     rho  divRho 
 4.2820 -0.4138  0.0334      NA      NA      NA      NA 
```
More accuracy can be obtained by increasing `N_steps` before running `amer_put.btree_prc`, or by increasing the `timeSteps` and `gridPoints` parameters to RQuantLib's `AmericanOption`.

From the RQuantLib documentation for `AmericanOption`:
```
Note that under the new pricing framework used in QuantLib, pricers do not provide analytics for all 'Greeks'. When “CrankNicolson” is selected, then at least delta, gamma and vega are available. With the default pricing engine of “BaroneAdesiWhaley”, no greeks are returned. 
```
Users of QuantLib may use this library for such cases... :)

### 3) Exotic American and European Options
Exotic options (with non-standard payoff functions) can be priced by specifying the Style as Other, and  providing the exotic's Payoff function (in place of the Strike price), when constructing the Terms of the contract. For examples, here are some digital/binary options: a *cash-or-nothing* call, which pays \\$1 if $S(T) >= K$, and an *asset-or-nothing* call, which pays the underlying asset if it exceeds the Strike price https://en.wikipedia.org/wiki/Binary_option:

In [8]:
// Cash-or-nothing call, see https://en.wikipedia.org/wiki/Binary_option
double payoff_cash_call(double s) {
    if (s >= K)
        return 1.0;
    else
        return 0.0;
};
Terms terms_cash_call{ Terms::Style::Euro, Terms::Type::Other, T, payoff_cash_call };
Option euro_cash_call(terms_cash_call, underlying);  // using previous underlying.
N_steps = 1000;
euro_cash_call.btree_prc(N_steps, r, get_greeks);
cout << "Euro Cash-or-Nothing Call Btree with N_steps: " << N_steps << '\n'
     << "r  Style    Class      K   T    S_o q sigma\n"
     << r << ' ' << euro_cash_call << ":\n"
     << "price: " << euro_cash_call.price() << '\n'
     << "delta: " << euro_cash_call.delta() << '\n' << "gamma: " << euro_cash_call.gamma() << '\n'
     << "vega: " << euro_cash_call.vega() << '\n' << "theta: " << euro_cash_call.theta() << '\n'
     << "rho: "  << euro_cash_call.rho() << endl;

// Calculate price analytically for comparison:
double d_1 = (std::log(S_o / K) + (r - q + sigma * sigma / 2) * T) / (sigma * std::sqrt(T));
double d_2 = d_1 - sigma * std::sqrt(T);
double BSM_price = std::exp(-r * T) * std::erfc(-d_2 / std::sqrt(2)) / 2;
cout << "Black-Scholes-Merton model price: " << BSM_price << endl;


Euro Cash-or-Nothing Call Btree with N_steps: 1000
r  Style    Class      K   T    S_o q sigma
0.1 European Other   -1 0.416667 50  0  0.4:
price: 0.504032
delta: 0.0295887
gamma: -0.000738311
vega: -0.27735
theta: 0.0501199
rho: 0.406393
Black-Scholes-Merton model price: 0.491943


In [9]:
// Asset-or-nothing call:
{   // This code in local block so xeus-cling accepts K via lambda capture..
T = 5/12.;
r = 0.1;
double K = 50.0;
std::function<double(double)> payoff_asset_call;
payoff_asset_call = [K](double s) { // Can specify Payoff_fn argument via lambda.
    if (s >= K)
        return s;
    else
        return 0.0;
};
Terms terms_asset_call{ Terms::Style::Euro, Terms::Type::Other, T, payoff_asset_call };
Option euro_asset_call(terms_asset_call, underlying);
N_steps = 10000;
euro_asset_call.btree_prc(N_steps, r, get_greeks);
cout << "\n European asset-or-nothing call with N_steps: " << N_steps << '\n'
     << "r   Style    Type     K   T     S_o q sigma\n"
     << r << ' ' << euro_asset_call << ":\n"     
     << "price: " << euro_asset_call.price() << '\n'
     << "delta: " << euro_asset_call.delta() << '\n' << "gamma: " << euro_asset_call.gamma() << '\n'
     << "theta: " << euro_asset_call.theta() << '\n' << "rho: " << euro_asset_call.rho() << '\n'
     << "vega: " << euro_asset_call.vega() << '\n';

BSM_price = S_o * std::exp(-q * T) * std::erfc(-d_1 / std::sqrt(2)) / 2;
cout << "Black-Scholes-Merton price: " << BSM_price << endl;
}


 European asset-or-nothing call with N_steps: 10000
r   Style    Type     K   T     S_o q sigma
0.1 European Other   -1 0.416667 50  0  0.4:
price: 30.9047
delta: 2.09503
gamma: -0.00483652
theta: -6.41739
rho: 30.7687
vega: -1.53656
Black-Scholes-Merton price: 30.7137


##### Note on pricing options with discontinuous payoffs
The above binary options each have a discontinuity in their respective payoff functions at the strike price `K`. The  *cash-or-nothing* call jumps from 0 to 1, while the *asset-or-nothing* call jumps from 0 to `K` at `K`. This may be expected to expose errors in numerical approximations (such as binary tree option pricing) due to round-off errors. These can be reduced by increasing `N_steps` and explains the use of a large value for pricing the *asset-or-nothing* call. Another step that was taken to reduce theses types of errors was to take advantage of the fact that the (theoretical) binomial tree of prices is recombining, so the price at the middle node for each time step is equal to the original asset price `S_o`. The original algorithm calculates node prices in terms of `O(N_steps)` multiplications of `S_o` by up and down factors `u` and `d`, which leads to prices of those middle nodes near, but sometimes not exactly equal to, `S_o`. Lines 170-175 of option_pricing.cc reset the middle node prices to their correct values. This algorithmic improvement led to more accurate option prices, and was actually initially motivated by drilling down into the original algorithm's behavior on its relatively poor pricing of a binary option. 

### 4) Monte Carlo pricing
Many options can be priced by Monte Carlo simulating the underlying asset's risk-neutral stochastic realizations. For options whose payoff depends only on the underlying asset price at time $T$(vs the entire path in time), we use $$S(T)=S(0)\cdot exp[(\mu - \sigma^2/2)T + \epsilon \sigma \sqrt{T}],$$ where $\mu=r-q$ is the expected risk-neutral return. Many draws, $\epsilon$, are made from the standard Normal distribution. The mean of their present values is our estimate of the current option price, and $\sqrt{\text{their variance}/ N}$ gives an estimate of the accuracy of this estimate.
As the value of an American option depends on forward estimates of its' value, they are not among the 'many options' referred to above; -use the binary tree for these. And, generally, contracts with early excercise provision can not be priced via the Monte Carlo pricer. We start by pricing a European put option. RQuantLib gives:
```
> EuropeanOption("put", 40.2, 43.0, 0.2, 0.07, 3/12, 0.32)
Concise summary of valuation for EuropeanOption 
  value   delta   gamma    vega   theta     rho  divRho 
 5.0560 -0.6723  0.0509  6.5787 -7.3699 -8.0206  6.7566
 ```

In [10]:
// Euro option via binary tree:
T = 3 / 12.0;
K = 43.0;
Terms terms_euro{ Terms::Style::Euro, Terms::Type::Put, T, K };
S_o = 40.2;
q = 0.2;
sigma = 0.32;
Asset underlying_euro{ S_o, q, sigma };
Option euro_put(terms_euro, underlying_euro);
r = 0.07;
N_steps = 1000;
get_greeks = true;
euro_put.btree_prc(N_steps, r, get_greeks);
cout << "On Binary pricing tree with N_steps: " << N_steps << '\n'
     << "r  Style    Class  K   T     S_o q sigma\n"
     << r << ' ' << euro_put << ":\n"
     << "price: " << euro_put.price() << '\n'
     << "delta: " << euro_put.delta() << '\n' << "gamma: " << euro_put.gamma() << '\n'
     << "vega: " << euro_put.vega() << '\n' << "theta: " << euro_put.theta() << '\n'
     << "rho: "  << euro_put.rho() << endl;

On Binary pricing tree with N_steps: 1000
r  Style    Class  K   T     S_o q sigma
0.07 European Put   43 0.25 40.2  0.2  0.32:
price: 5.05638
delta: -0.672275
gamma: 0.0509099
vega: 6.60733
theta: -7.37166
rho: -8.01976


In [11]:
// Same Euro option via Monte Carlo pricer:
int N_sims = 10000;
bool antithetic = true;
get_greeks = true;
double sd_prc{0.0};
euro_put.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nEuro put with Monte Carlo pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
     << "r  Style    Class  K   T     S_o q sigma\n"
     << r << ' ' << euro_put << ":\n"
     << "price: " << euro_put.price() << ", std_dev: " << sd_prc << '\n'
     << "delta: " << euro_put.delta() << '\n' << "gamma: " << euro_put.gamma() << '\n'
     << "vega: " << euro_put.vega() << '\n' << "theta: " << euro_put.theta() << '\n'
     << "rho: "  << euro_put.rho() << endl;


Euro put with Monte Carlo pricer with N_sims: 10000, N_steps: 1000
r  Style    Class  K   T     S_o q sigma
0.07 European Put   43 0.25 40.2  0.2  0.32:
price: 5.05728, std_dev: 0.0102288
delta: -0.66921
gamma: 0.0544345
vega: 6.62198
theta: -7.38657
rho: -7.99056


##### Some notes on the Monte Carlo pricer
As the results produced are based on random draws, each run of the Monte Carlo pricer will produce a different stream of random numbers and results. This behavior can be overridden by giving the random number generator underlying the Normally distributed draws a seed which acts as the starting point for the (psuedo-) random stream of numbers. Un-commment lines 256 and 257 of option_pricing.cc, and comment out the 2 lines that follow, to create deterministic streams for reproducing results between runs such as for de-bugging.

As the option price and values of the greeks are found by averaging N_sims runs, each of which generates a value for these quantities according to the risk-neutral stochastic realizations, the accuracy of these will tend to improve as N_sims increases -the standard deviation of these means will be close to the standard deviation of the simulated distributions of the original quantities(price or greeks) divided by the square root of N_sims - this follows from the Central Limit Theorem. This std_dev is calculated as just described and reported for the option price. By adding 2 zeroes to the N_sims, one more order of magnitude in price accuracy is obtained. It would be straight forward but a bit tedious to add code to estimate the std_dev of the greeks in the same manner; currently these can be estimated by observing their fluctuations from run to run (assuming random seeding of the random number generator!).

#### Variance Reduction -Antithetic Sampling
The variance of the distribution of simulated payoffs, or of simulated greeks, can be reduced using the technique of antithetic sampling. By setting the bool argument to the MC pricer, `antithetic`, to true, the payoff calculated from any particular random draw is paired with the payoff calculated from the antithetic draw obtained by replacing $\epsilon$ with $-\epsilon$. As their means are identical, and, for payoffs that tend to be monotonic with $S(T)$ the payoffs among these pairs will be negatively correlated, the average of these pairs will give the desired mean and have a smaller variance. For the above Euro Put option, toggling `antithetic` to `true` from `false` reduces the standard deviation of the estimated price by about a factor 4 (corresponding to a correlation between the antithetic Payoffs of -0.875). This factor depends on the shape (especially the monotonicity as mentioned) of the Payoff function in the region being sampled by the simulated $S(T)$: 

* Antithetic sampling does **not** improve the performance for the following at the money straddle option as the payoff is not monotonic in $S(T)$ (it's actually symmetric about `K`, and the correlation between the antithetically paired Payoffs is close to $+1$):


In [12]:
// At the money Straddle option via Monte Carlo pricer:
double straddle_payoff(double S) {
    if(S > 40.2)
        return S - 40.2;
    else
        return 40.2 - S;
}
Terms terms_straddle{ Terms::Style::Euro, Terms::Type::Other, T, straddle_payoff};
Option euro_straddle(terms_straddle, underlying_euro);
N_sims = 10000;
antithetic = false;  // setting to true does not decrease std_dev of price found.
get_greeks = true;
sd_prc = 0.0;
euro_straddle.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nEuro Straddle with MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
     << "r    Style    Class    K T    S_o  q  sigma\n"
     << r << ' ' << euro_straddle << ":\n"
     << "price: " << euro_straddle.price() << ", std_dev: " << sd_prc << '\n'
     << "delta: " << euro_straddle.delta() << '\n' << "gamma: " << euro_straddle.gamma() << '\n'
     << "vega: " << euro_straddle.vega() << '\n' << "theta: " << euro_straddle.theta() << '\n'
     << "rho: "  << euro_straddle.rho() << endl;


Euro Straddle with MC pricer with N_sims: 10000, N_steps: 1000
r    Style    Class    K T    S_o  q  sigma
0.07 European Other   -1 0.25 40.2  0.2  0.32:
price: 5.04656, std_dev: 0.0371802
delta: -0.0986562
gamma: 0.116326
vega: 15.0834
theta: -9.83639
rho: -2.24041


* Antithetic sampling is quite effective for this deep in the money Call Option as the payoff is strongly monotonic (delta near 1, gamma near 0):

In [13]:
// Deep in the money call where antithetic sampling decreases price std_dev by 9x:
T = 3 / 12.0;
K = 40.0;
Terms terms_euro_call{ Terms::Style::Euro, Terms::Type::Call, T, K };
S_o = 60;
q = 0.2;
sigma = 0.32;
Asset underlying_euro_call{ S_o, q, sigma };
Option euro_call(terms_euro_call, underlying_euro_call);
r = 0.07;
N_steps = 1000;
get_greeks = true;
N_sims = 100000;
antithetic = true;
sd_prc = 0.0;
euro_call.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nEuro call with MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
     << "r    Style    Class  K T   S_o  q  sigma\n"
     << r << ' ' << euro_call << ":\n"
     << "price: " << euro_call.price() << ", std_dev: " << sd_prc << '\n'
     << "delta: " << euro_call.delta() << '\n' << "gamma: " << euro_call.gamma() << '\n'
     << "vega: " << euro_call.vega() << '\n' << "theta: " << euro_call.theta() << '\n'
     << "rho: "  << euro_call.rho() << endl;


Euro call with MC pricer with N_sims: 100000, N_steps: 1000
r    Style    Class  K T   S_o  q  sigma
0.07 European Call  40 0.25 60  0.2  0.32:
price: 17.7914, std_dev: 0.00372732
delta: 0.943782
gamma: 0.00199947
vega: 0.611413
theta: 8.23312
rho: 9.7091


#### Path dependent payoffs -Asian Options
Asian options have payoffs which depend on the actual price path of the underlying asset for $t$ in $[0,T]$. For example, an AsianPrice call option has a payoff $max(\tilde{S} - K, 0) $, where $\tilde{S}$ is either the arithmetic or geometric average of $S(t)$ along the path. One effect of this is to make the option's payoff less sensitive to large moves in $S$ at expiry.


In [14]:
// Price an AsianPrc Put option with Arithmetic path average price:
T = 6 / 12.0;
K = 50.0;
Terms terms_asianprc_put{ Terms::Style::AsianPrc, Terms::Type::Put, T, K };
S_o = 49.0;
q = 0.0;
sigma = 0.3;
Asset underlying_asian{ S_o, q, sigma };
Option asianprc_put(terms_asianprc_put, underlying_asian);
r = 0.07;
N_steps = 1000;
N_sims = 10000;
antithetic = true;
get_greeks = true;
sd_prc = 0.0;
asianprc_put.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nAsianPrc put with Monte Carlo pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
     << "r    Style    AvgType   Class  K  T  S_o q sigma\n"
     << r << ' ' << asianprc_put << ":\n"
     << "price: " << asianprc_put.price() << ", std_dev: " << sd_prc << '\n'
     << "delta: " << asianprc_put.delta() << '\n' << "gamma: " << asianprc_put.gamma() << '\n'
     << "theta: " << asianprc_put.theta() << '\n' << "rho: " << asianprc_put.rho() << '\n'
     << "vega: " << asianprc_put.vega() << endl;



AsianPrc put with Monte Carlo pricer with N_sims: 10000, N_steps: 1000
r    Style    AvgType   Class  K  T  S_o q sigma
0.07 AsianPrc Arithmetic Put   50 0.5 49  0  0.3:
price: 2.42792, std_dev: 0.0150047
delta: -0.480824
gamma: 0.0635803
theta: -1.36682
rho: -6.99862
vega: 7.86196


This agrees well with the result from RQuantLib:
```
> AsianOption("arithmetic", "put", 49.0, 50.0, 0.0, .07, 0.5, 0.3, length=0.5, fixings=1000)
Concise summary of valuation for AsianOption 
 value  delta  gamma   vega  theta    rho divRho 
2.4287     NA     NA     NA     NA     NA     NA 
```
The documentation explaining the `length` and `fixings` arguments to `AsianOption` is not very definite, but I found that setting `length == maturity` and `fixings == N_steps` seemed to work. Unfortunately `AsianOption` does not calculate greeks for `averageType == arithmetic`; here is the corresponding option with `geometric` price averaging where those last two fields don't matter (for some reason, and behaving as if you can not adjust the number of time steps, and the geometric averaging occurs from time 0 to $T$ as I do it.):
```
> AsianOption("geometric", "put", 49.0, 50.0, 0.0, .07, 0.5, 0.3, length=0.5, fixings=10000)
Concise summary of valuation for AsianOption 
  value   delta   gamma    vega   theta     rho  divRho 
 2.5085 -0.4861  0.0651  8.4080 -1.5131 -7.2095  5.9553 
> AsianOption("geometric", "put", 49.0, 50.0, 0.0, .07, 0.5, 0.3, length=0.05, fixings=10)
Concise summary of valuation for AsianOption 
  value   delta   gamma    vega   theta     rho  divRho 
 2.5085 -0.4861  0.0651  8.4080 -1.5131 -7.2095  5.9553 
> AsianOption("geometric", "put", 49.0, 50.0, 0.0, .07, 0.5, 0.3)
Concise summary of valuation for AsianOption 
  value   delta   gamma    vega   theta     rho  divRho 
 2.5085 -0.4861  0.0651  8.4080 -1.5131 -7.2095  5.9553 
```


In [15]:
// Price an AsianPrc Put option with Geometric path average price:
Terms terms_asianprc_put_geo{ Terms::Style::AsianPrc, Terms::Type::Put, T, K, Terms::AvgType::Geom };
Option asianprc_put_geo(terms_asianprc_put_geo, underlying_asian);
N_steps = 1000;
N_sims = 10000;
antithetic = true;
get_greeks = true;
sd_prc = 0.0;
asianprc_put_geo.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nAsianPrc Put Geometric path avgeraging with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
     << "r    Style    AvgType   Class  K  T  S_o q sigma\n"
     << r << ' ' << asianprc_put_geo << ":\n"
     << "price: " << asianprc_put_geo.price() << ", std_dev: " << sd_prc << '\n'
     << "delta: " << asianprc_put_geo.delta() << '\n' << "gamma: " << asianprc_put_geo.gamma() << '\n'
     << "theta: " << asianprc_put_geo.theta() << '\n' << "rho: " << asianprc_put_geo.rho() << '\n'
     << "vega: " << asianprc_put_geo.vega() << endl;



AsianPrc Put Geometric path avgeraging with N_sims: 10000, N_steps: 1000
r    Style    AvgType   Class  K  T  S_o q sigma
0.07 AsianPrc Geometric Put   50 0.5 49  0  0.3:
price: 2.52365, std_dev: 0.0153097
delta: -0.485835
gamma: 0.0623526
theta: -1.46552
rho: -7.21427
vega: 8.45535


#### Arbitrary Path Averaging and Payoffs
I have defined contracts which depend on some type of path average/accumulation, and having no provision for early excercise, as Asian Options. Asian Price`(Terms::Style==AsianPrc)` options payoff at expiration (no early excercise) $max(\tilde{S}-K, 0)$ if `Terms::Type==Call`, and $max(K-\tilde{S}, 0)$ if `Terms::Type==Put`, where $\tilde{S}$ is either the arithmetic `(Terms::AvgType==Arith)` or geometric `(Terms::AvgType==Geom)` average along the price path $S(t), t\in(0, T)$. Asian Strike (call and put) `(Terms::Style==AsianStrike)` options payoff at expiration in the same way as the Asian Price options except with the replacements $\tilde{S}\rightarrow S(T)$ and $K\rightarrow\tilde{S}$.

Non-path dependent European and American options can be priced as shown above in the Section "Exotic American and European Options". There we needed to specify a payoff function and give the option `Terms::Style==Other`. Europeans can be priced via binary tree or by Monte Carlo, Americans require the binary tree as the Monte Carlo does not account for early excercise.

Asian contracts which depend on a path average which is not the standard arithmetic or gemometric average, or whose payoff is not of the standard vanilla call or put in terms of those averages, are contracts with `Terms::Style==AsianExotic` where we must specify both the payoff function and the path averaging function. This is illustrated here with 4 different contracts of increasing complexity. Enjoy!

In [16]:
// Price an AsianExotic, arithmetic path avg is the payoff:
std::function<double(double, double)> path_payoff{
    [](double path_avg, double) { return path_avg; }
};

T = 6 / 12.0;
Terms terms_asianex_arith{ Terms::Style::AsianExotic, Terms::Type::Other, T, path_payoff,
                               Terms::arith_path_accum_fn, 0.0 };
S_o = 49.0;
q = 0.01;
sigma = 0.3;
Asset underlying_asian2{ S_o, q, sigma };
Option asianex_arith(terms_asianex_arith, underlying_asian2);
r = 0.07;
N_steps = 1000;
N_sims = 10000;
antithetic = true;
get_greeks = true;
sd_prc = 0.0;
asianex_arith.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nAsianExotic arithmetic_avg payoff MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
    << "r    Style     AvgType Class  K  T  S_o  q  sigma\n"
    << r << ' ' << asianex_arith << ":\n"
    << "price: " << asianex_arith.price() << ", std_dev: " << sd_prc << '\n'
    << "delta: " << asianex_arith.delta() << '\n' << "gamma: " << asianex_arith.gamma() << '\n'
    << "theta: " << asianex_arith.theta() << '\n' << "rho: " << asianex_arith.rho() << '\n'
    << "vega: " << asianex_arith.vega() << endl;


AsianExotic arithmetic_avg payoff MC pricer with N_sims: 10000, N_steps: 1000
r    Style     AvgType Class  K  T  S_o  q  sigma
0.07 AsianExotic Other Other   - 0.5 49  0.01  0.3:
price: 48.0366, std_dev: 0.00631772
delta: 0.980338
gamma: -8.19299e-13
theta: 1.88808
rho: -11.935
vega: 0.0288004


In [19]:
// Define tanh path accumulation function used below:
std::function<void(int, double, double&)> path_accum_tanh;
double S_o = 49.0;

path_accum_tanh = [S_o](int idx, double S, double& accum) { 
    accum += (std::tanh(S / S_o) - accum) / idx; 
};

In [20]:
// AsianExotic pays off tanh average:
T = 6/12.0;
Terms terms_asian_tanh_call{ Terms::Style::AsianExotic, Terms::Type::Other, T, path_payoff,
                             path_accum_tanh, 0.0 };
Option asian_tanh_call(terms_asian_tanh_call, underlying_asian2);
r = 0.07;
N_steps = 1000;
N_sims = 10000;
antithetic = true;
get_greeks = true;
sd_prc = 0.0;
asian_tanh_call.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
cout << "\nAsianExotic tanh_path_avg 0_call MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
    << "r    Style     AvgType Class  K  T  S_o  q  sigma\n"
    << r << ' ' << asian_tanh_call << ":\n"
    << "price: " << asian_tanh_call.price() << ", std_dev: " << sd_prc << '\n'
    << "delta: " << asian_tanh_call.delta() << '\n' << "gamma: " << asian_tanh_call.gamma() << '\n'
    << "theta: " << asian_tanh_call.theta() << '\n' << "rho: " << asian_tanh_call.rho() << '\n'
    << "vega: " << asian_tanh_call.vega() << endl;


AsianExotic tanh_path_avg 0_call MC pricer with N_sims: 10000, N_steps: 1000
r    Style     AvgType Class  K  T  S_o  q  sigma
0.07 AsianExotic Other Other   - 0.5 49  0.01  0.3:
price: 0.734559, std_dev: 2.68874e-05
delta: 0.00808026
gamma: -0.000252683
theta: 0.0532506
rho: -0.268958
vega: -0.0451952


In [21]:
   // abs(sin(tanh_path_avg)) payoff function:
    std::function<double(double, double)> path_payoff_sin_tanh_abs{
        [](double path_avg, double S) { return std::abs(std::sin((path_avg - std::tanh(S)) / (std::tanh(S) / 10.0))); }
    };

    Terms terms_asian_tanh_sin_abs{ Terms::Style::AsianExotic, Terms::Type::Other, T, path_payoff_sin_tanh_abs,
                                 path_accum_tanh, 0.0 };
    Option asian_tanh_sin_abs{ terms_asian_tanh_sin_abs, underlying_asian2 };
    antithetic = true;
    get_greeks = true;
    sd_prc = 0.0;
    asian_tanh_sin_abs.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
    cout << "\nAsianExotic abs(sin(tanh_path_avg)) MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
        << "r    Style     AvgType Class  K  T  S_o  q  sigma\n"
        << r << ' ' << asian_tanh_sin_abs << ":\n"
        << "price: " << asian_tanh_sin_abs.price() << ", std_dev: " << sd_prc << '\n'
        << "delta: " << asian_tanh_sin_abs.delta() << '\n' << "gamma: " << asian_tanh_sin_abs.gamma() << '\n'
        << "theta: " << asian_tanh_sin_abs.theta() << '\n' << "rho: " << asian_tanh_sin_abs.rho() << '\n'
        << "vega: " << asian_tanh_sin_abs.vega() << endl;



AsianExotic abs(sin(tanh_path_avg)) MC pricer with N_sims: 10000, N_steps: 1000
r    Style     AvgType Class  K  T  S_o  q  sigma
0.07 AsianExotic Other Other   - 0.5 49  0.01  0.3:
price: 0.615573, std_dev: 0.000579606
delta: 0.0419687
gamma: -0.00142275
theta: 0.0521811
rho: 0.207824
vega: -0.238629


In [22]:
   // AsianExotic sin(tanh_path_avg) payoff:
    std::function<double(double, double)> path_payoff_sin_tanh{
        [](double path_avg, double S) { return std::sin((path_avg - std::tanh(S)) / (std::tanh(S) / 10.0)); }
    };

    Terms terms_asian_tanh_sin{ Terms::Style::AsianExotic, Terms::Type::Other, T, path_payoff_sin_tanh,
                                 path_accum_tanh, 0.0 };
    Option asian_tanh_sin{ terms_asian_tanh_sin, underlying_asian2 };
    antithetic = true;
    get_greeks = true;
    sd_prc = 0.0;
    asian_tanh_sin.MC_prc(N_sims, N_steps, r, get_greeks, antithetic, sd_prc);
    cout << "\nAsianExotic sin(tanh_path_avg) MC pricer with N_sims: " << N_sims << ", N_steps: " << N_steps << '\n'
        << "r    Style     AvgType Class  K  T  S_o  q  sigma\n"
        << r << ' ' << asian_tanh_sin << ":\n"
        << "price: " << asian_tanh_sin.price() << ", std_dev: " << sd_prc << '\n'
        << "delta: " << asian_tanh_sin.delta() << '\n' << "gamma: " << asian_tanh_sin.gamma() << '\n'
        << "theta: " << asian_tanh_sin.theta() << '\n' << "rho: " << asian_tanh_sin.rho() << '\n'
        << "vega: " << asian_tanh_sin.vega() << endl;



AsianExotic sin(tanh_path_avg) MC pricer with N_sims: 10000, N_steps: 1000
r    Style     AvgType Class  K  T  S_o  q  sigma
0.07 AsianExotic Other Other   - 0.5 49  0.01  0.3:
price: -0.583391, std_dev: 0.00109367
delta: -0.0537609
gamma: 0.00540238
theta: -0.174151
rho: -0.367473
vega: 0.725033
